In [1]:
#!pip install -U skorch
!pip install efficientnet_pytorch

In [2]:
import os
import shutil
import hashlib

import numpy as np
import pandas as pd
import cv2

%matplotlib inline
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from PIL import Image
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import albumentations as A
from albumentations.pytorch import ToTensor

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
DIR_IMAGES = "/content/drive/My Drive/train_data/images/"
DIR_DF = "/content/drive/My Drive/train_data/train.csv"

SIZE = 224
FOLDS = 5
BATCH_SIZE = 32
SEED = 1080
MODEL_NAME = 'efficientnet-b4'
NUM_CLASSES = 2

### Check the amount of data for each class


In [9]:
train_data = pd.read_csv('/content/drive/My Drive/train_data/train.csv')
train_data['emergency_or_not'].value_counts()

0    965
1    681
Name: emergency_or_not, dtype: int64

In [10]:
train_data.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


### Create Image metadata

In [11]:
def calculate_hash(im):
    md5 = hashlib.md5()
    md5.update(np.array(im).tostring())
    
    return md5.hexdigest()


def get_image_meta(image_id, image_src, dataset='train'):
    im = Image.open(image_src)
    extrema = im.getextrema()

    meta = {
        'image_id': image_id,
        'dataset': dataset,
        'hash': calculate_hash(im),
        'r_min': extrema[0][0],
        'r_max': extrema[0][1],
        'g_min': extrema[1][0],
        'g_max': extrema[1][1],
        'b_min': extrema[2][0],
        'b_max': extrema[2][1],
        'height': im.size[0],
        'width': im.size[1],
        'format': im.format,
        'mode': im.mode
    }
    return meta

### Dataset Class

In [12]:
class VehicleDataset(Dataset):
  """ Emergency Vehicles Dataset. """
  def __init__(self, dataframe, root_dir, transform=None, train=True):
    """ 
    Parameters:
      csv_file(string): Path to the csv file containing the labels.
      root_dir(string): Path to the folder that contains the images.
      transforms(callable): Optional transforms to be applied on a sample."""
    self.vehicles_frame = dataframe
    self.root_dir = root_dir
    self.transform = transform
    self.train = train

  def __len__(self):
    return(self.vehicles_frame.shape[0])
  
  def __getitem__(self, idx):
    img_name = self.root_dir + self.vehicles_frame['image_names'].iloc[idx]
    image = cv2.imread(img_name, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transformed = self.transform(image=image)
    image = transformed['image']
    if(self.train):
      labels = torch.tensor(self.vehicles_frame['emergency_or_not'].iloc[idx])
    else:
      labels = idx
    return(image, labels)

### Define Transforms

In [13]:
train_transform = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(rotate_limit=0.5, p=0.8),

    # Pixels
    A.OneOf([
        A.IAAEmboss(p=1.0),
        A.IAASharpen(p=1.0),
        A.Blur(p=1.0),
    ], p=0.5),

    A.OneOf([
        A.ElasticTransform(p=1.0),
        A.IAAPiecewiseAffine(p=1.0)
    ], p=0.5),
    
    A.Normalize(p=1.0),
    ToTensor(),
])

transforms_valid = A.Compose([
    A.Resize(height=SIZE, width=SIZE, p=1.0),
    A.Normalize(p=1.0),
    ToTensor(),
])

### Test Dataset Class



In [ ]:
# train_dataset = VehicleDataset(train_data, DIR_IMAGES, train_transform)
# train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# images, labels = next(iter(train_dataloader))
# del train_dataloader, train_dataset

###  V4 - EfficientNet Model


In [30]:
model = EfficientNet.from_pretrained(MODEL_NAME)

input_features = model._fc.in_features
model._fc = nn.Sequential(nn.Linear(input_features, 1000, bias=True),
                          nn.ReLU(),
                          nn.Dropout(p=0.5),
                          nn.Linear(1000, NUM_CLASSES, bias = True),
                          nn.Softmax(dim=1))

Loaded pretrained weights for efficientnet-b4


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PATH = 'init.pth'
torch.save(model.state_dict(), PATH)
model.to(device); 

In [34]:
def compute_val_metrics(model, valid_loader):
  model.eval()
  validation_loss = 0
  correct = 0
  size = 0
  with torch.no_grad():
    for inputs, labels in valid_loader:
      # Move data to the Cuda.
      inputs, labels = inputs.to(device), labels.float().to(device)
      # Compute the model outputs
      output= model.forward(inputs)
      # Compute loss function
      loss = criterion(output[:, -1], labels)
      validation_loss += loss
      # Compute Result
      result = output.max(axis=1)[1]
      # Find the number of correctly classifid examples
      correct += torch.sum(torch.eq(result.type(labels.type()), labels)).item()
      # Find the total number of samples
      size += labels.size(0)
  return(validation_loss, (correct * 100 /size))


def train_one_epoch(model, optmizer, train_loader, valid_loader):
  train_loss = 0
  model.train()
  for inputs, labels in train_dataloader:
    inputs, labels = inputs.to(device), labels.float().to(device)

    optimizer.zero_grad()
  
    logps = model.forward(inputs)
    loss = criterion(logps[:, -1], labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  validation_loss, valid_accuracy = compute_val_metrics(model, valid_loader)
  scheduler.step(validation_loss)
  _, train_accuracy = compute_val_metrics(model, train_loader)
  return(train_loss, validation_loss, valid_accuracy, train_accuracy)

In [35]:
folds = KFold(n_splits=5, shuffle=True, random_state=SEED)

In [36]:
from collections import defaultdict
# Dicts to store training and test accuracy
valid_accuracy_dict = defaultdict(list)
train_accuracy_dict = defaultdict(list)
for i_fold, (train_idx, valid_idx) in enumerate(folds.split(train_data)):
  epochs = 30
  # Validation DataFrame
  valid = train_data.iloc[valid_idx]
  valid.reset_index(drop=True, inplace=True)
  # Training DataFrame
  train = train_data.iloc[train_idx]
  train.reset_index(drop=True, inplace=True)
  # Train DataLoader
  train_dataset = VehicleDataset(train, DIR_IMAGES, train_transform)
  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
  # Valid Dataloder
  valid_dataset = VehicleDataset(valid, DIR_IMAGES, transforms_valid)
  valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
  # Reset model and loss functions every iteration
  criterion = nn.BCELoss()
  model.load_state_dict(torch.load(PATH)) 
  model.to(device);
  parameters =  list(model._fc.parameters())
  optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.3, patience=3)
  for epoch in range(epochs):
    print(f"Fold {i_fold}")
    train_loss, valid_loss, valid_accuracy, train_accuracy = train_one_epoch(model, optimizer, train_dataloader, valid_dataloader)
    train_accuracy_dict[i_fold].append(train_accuracy)
    valid_accuracy_dict[i_fold].append(valid_accuracy)
    print(f"Epoch {epoch+1} / {epochs}.. "
            f"Train loss: {train_loss:.3f}.. "
            f"Val Loss:{valid_loss:.3f}.. "
            f"Val Accuracy:{valid_accuracy}.."
            f"Train Accuracy:{train_accuracy}")
  torch.save(model.state_dict(), '/content/drive/My Drive/model'+str(i_fold)+'.pth')
  break

Fold 0
Epoch 1 / 30.. Train loss: 21.086.. Val Loss:2.581.. Val Accuracy:91.81818181818181..Train Accuracy:87.23404255319149
Fold 0
Epoch 2 / 30.. Train loss: 13.801.. Val Loss:4.313.. Val Accuracy:88.48484848484848..Train Accuracy:83.8145896656535
Fold 0
Epoch 3 / 30.. Train loss: 11.325.. Val Loss:6.460.. Val Accuracy:82.42424242424242..Train Accuracy:80.09118541033435
Fold 0
Epoch 4 / 30.. Train loss: 9.917.. Val Loss:4.167.. Val Accuracy:85.45454545454545..Train Accuracy:89.8176291793313
Fold 0
Epoch 5 / 30.. Train loss: 11.260.. Val Loss:8.801.. Val Accuracy:63.63636363636363..Train Accuracy:68.16109422492401
Fold 0
Epoch 6 / 30.. Train loss: 11.767.. Val Loss:2.424.. Val Accuracy:91.81818181818181..Train Accuracy:92.70516717325228
Fold 0
Epoch 7 / 30.. Train loss: 6.395.. Val Loss:2.184.. Val Accuracy:94.24242424242425..Train Accuracy:95.44072948328268
Fold 0
Epoch 8 / 30.. Train loss: 5.122.. Val Loss:2.330.. Val Accuracy:94.84848484848484..Train Accuracy:97.11246200607903
Fold 

In [38]:
 test_transform = A.Compose([
                                A.Resize(height=SIZE, width=SIZE, p=1.0),
                                A.HorizontalFlip(p=0.5),
                                A.VerticalFlip(p=0.5),
                                A.ShiftScaleRotate(rotate_limit=25.0, p=0.7),
                                A.Normalize(p=1.0),
                                ToTensor()])

### Evaluate accuracy on the training set


In [30]:
test_train_dataset = VehicleDataset(train_data, DIR_IMAGES, test_transform)
test_train_dataloader = DataLoader(test_train_dataset, batch_size=64, shuffle=True)

In [31]:
_, accuracy = compute_val_metrics(model, test_train_dataloader)

In [32]:
accuracy

95.80801944106926

### Sumbit the model and report test accuracy


In [39]:
test_df = pd.read_csv('/content/drive/My Drive/train_data/test_vc2kHdQ.csv')
test_df['emergency_or_not'] = 0
test_dataset = VehicleDataset(test_df, DIR_IMAGES, test_transform, False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [40]:
test_df['results'] = 0
model.eval()
with torch.no_grad():
  model.load_state_dict(torch.load('/content/drive/My Drive/model0.pth'))
  for i in range(5):

    for image_id, index in test_dataloader:
      image_id = image_id.cuda()
      output = model(image_id)
      test_df['results'].loc[index] += output[:, -1].cpu().numpy()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [41]:
test_df['emergency_or_not'] = (test_df['results'] > FOLDS/2).astype(int)
test_df.to_csv('results.csv')

,image_names,emergency_or_not,results
0,1960.jpg,0,3.051052e-06
1,668.jpg,1,4.997466e+00
2,2082.jpg,0,1.083528e-07
3,808.jpg,1,4.998066e+00
4,1907.jpg,0,6.805246e-09
...,...,...,...
701,674.jpg,1,4.999930e+00
702,1027.jpg,0,6.064989e-05
703,447.jpg,1,4.998716e+00
704,2176.jpg,0,1.481521e-05
